In [16]:
# USER INPUT
city_name = 'hayward'
url = 'https://hayward.legistar.com/Calendar.aspx'
root_url = 'https://hayward.legistar.com/'

In [17]:
# imports
import pandas as pd
import numpy as np
from urllib import parse
from urllib import request
import bs4 as bs

import selenium as se
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm

# helper functions
def get_page_links(driver):
    pagelinks_xpath = "//td[@class='rgPagerCell NumericPages']/div[1]/a"
    pagelinks = driver.find_elements_by_xpath(pagelinks_xpath)
    pagelinks = pagelinks[:int(len(pagelinks)/2)]
    return [l.text for l in pagelinks], pagelinks

def extract_table(page_source, table_id):
    # find table in page
    soup = bs.BeautifulSoup(page_source)
    table = soup.select(table_id)[0]
    num_cols = int(table.td.get('colspan'))

    # extract column headers
    header_data = [''.join(cell.stripped_strings) for cell in table.find_all('th')]
    header_data = [h for h in header_data if h!='Data pager']
    assert(len(header_data)==num_cols)

    # extract text and URL data from table
    text_data, url_data = [], []
    for row in table.find_all('tr'):
        row_text, row_url = [], []
        for td in row.find_all('td'):
            row_text.append(''.join(td.stripped_strings))
            if td.find('a') and (td.a.get('href') is not None):
                row_url.append(root_url+td.a.get('href'))
            else:
                row_url.append(np.nan)
            if len(row_text)==num_cols and len(row_url)==num_cols:
                text_data.append(row_text)
                url_data.append(row_url)
                
    # turn into dataframe
    num_cols = table.td.get('colspan')
    text_df = pd.DataFrame(text_data, columns=header_data)
    url_df = pd.DataFrame(url_data, columns=header_data)
    df = pd.merge(text_df, url_df, left_index=True, right_index=True, suffixes=(' Text', ' URL'))
    
    return df

In [3]:
# launch driver
driver = Firefox()
driver.get(url)

In [4]:
# MANUAL: select date range dropdown  to 'All Years'

In [4]:
# click through pages and save html
c = 1
page_data = []
while True:
    pages, pagelinks = get_page_links(driver)
    try:
        # click on the integer we want
        i = pages.index(str(c))
        link = pagelinks[i]
    except:
        # if it's not there and the list ends with '...', click on '...'
        if pages[-1]=='...':
            link = pagelinks[-1]
        # if it's not there and the list starts with '...', we are done.
        else:
            break
    link.click()
    input('Scrape page {}?'.format(c))
    page_data.append(driver.page_source)
    c += 1

Scrape page 1?
Scrape page 2?
Scrape page 3?
Scrape page 4?
Scrape page 5?
Scrape page 6?
Scrape page 7?
Scrape page 8?
Scrape page 9?
Scrape page 10?
Scrape page 11?
Scrape page 12?
Scrape page 13?


In [6]:
# page_dfs[-1]

'<html xmlns="http://www.w3.org/1999/xhtml" lang="en"><head id="ctl00_Head1"><script type="text/javascript" src="https://bam.nr-data.net/1/2fdd6b8d1a?a=50175531&amp;v=1130.54e767a&amp;to=YwdbYEZTVxYHABALW1pNeGdkHVoECgYKBlVGTFhHREo%3D&amp;rst=1954&amp;ref=https://hayward.legistar.com/Calendar.aspx&amp;ap=790&amp;be=1100&amp;fe=1898&amp;dc=1646&amp;perf=%7B%22timing%22:%7B%22of%22:1572124464995,%22n%22:0,%22u%22:1082,%22ue%22:1092,%22f%22:12,%22dn%22:12,%22dne%22:12,%22c%22:12,%22s%22:0,%22ce%22:12,%22rq%22:18,%22rp%22:1071,%22rpe%22:1071,%22dl%22:1082,%22di%22:1607,%22ds%22:1645,%22de%22:1703,%22dc%22:1898,%22l%22:1898,%22le%22:1918%7D,%22navigation%22:%7B%7D%7D&amp;at=T0BYFg5JGwsUTRAQXUQrXRYOEHhWUlVcVQcHUQpyBAZ4Rxse&amp;jsonp=NREUM.setToken"></script><script type="text/javascript" src="https://m.addthis.com/live/red_lojson/300lo.json?si=5db4b732f72dcd72&amp;bkl=0&amp;bl=1&amp;pdt=1096&amp;sid=5db4b732f72dcd72&amp;pub=legistarinsite&amp;rev=v8.28.0-wp&amp;ln=en&amp;pc=men&amp;cb=0&amp;a

In [18]:
# extract table data
table_id = '#ctl00_ContentPlaceHolder1_gridCalendar_ctl00'
page_dfs = [extract_table(page, table_id) for page in tqdm(page_data)]


100%|██████████| 13/13 [00:03<00:00,  3.67it/s]

In [19]:
page_dfs[-1].loc[0,'Agenda URL']

'https://hayward.legistar.com/View.ashx?M=A&ID=493104&GUID=8E091B49-5FDF-46AA-A729-E465D282316C'

In [20]:
data = pd.concat(page_dfs)
len(data)

1289

In [21]:
data.columns

Index(['Name Text', 'Meeting Date Text', ' Text', 'Meeting Time Text',
       'Meeting Location Text', 'Meeting Details Text',
       'Staff/ProjectApplicant Presentations Text', 'Agenda Text',
       'Action Minutes Text', 'Documents ReceivedAfter Published Agenda Text',
       'Official Minutes Text', 'Video Text', 'Name URL', 'Meeting Date URL',
       ' URL', 'Meeting Time URL', 'Meeting Location URL',
       'Meeting Details URL', 'Staff/ProjectApplicant Presentations URL',
       'Agenda URL', 'Action Minutes URL',
       'Documents ReceivedAfter Published Agenda URL', 'Official Minutes URL',
       'Video URL'],
      dtype='object')

In [22]:
save_path = '../data/scraping/scraped_tables/{}.csv'.format(city_name)
data.to_csv(save_path)

In [12]:
# # generate results
# submit_button_id = 'ctl00_ContentPlaceHolder1_btnSearch'
# button = driver.find_element_by_id(submit_button_id)
# button.submit()